# Imports

In [27]:
import importlib
import utils.biopython_tools as bptools
import json
import pandas as pd

In [28]:
importlib.reload(bptools)

<module 'utils.biopython_tools' from '/home/markus/projects/riff_diff/ensemble_evaluator/ensemble_evaluator/utils/biopython_tools.py'>

In [9]:
motif_d = {"A": [1, 2, 3, 4, 5], "B": [6, 7, 8, 9]}
def convert_motifdict_to_list(motif_dict: dict) -> list: 
    return [(chain, res) for chain in motif_dict for res in motif_dict[chain]]

print(convert_motifdict_to_list(motif_d))

[('A', 1), ('A', 2), ('A', 3), ('A', 4), ('A', 5), ('B', 6), ('B', 7), ('B', 8), ('B', 9)]


# Test replacement

In [18]:
# get motif dicts:
motif_df = pd.read_json("/home/markus/projects/rfdiffusion_testing/motif_replacement/motif_res.json")
pose_motif = motif_df.loc["D33-C34-B26-A44_0001_0001"]["motif_residues"]

In [12]:
pose_p = "/home/markus/projects/rfdiffusion_testing/motif_replacement/D33-C34-B26-A44_0001_0001.pdb"
motif_pose_p = "/home/markus/projects/rfdiffusion_testing/motif_replacement/ref_fragment/D33-C34-B26-A44_0001_0001.pdb"

In [46]:
importlib.reload(bptools)
bptools.replace_motif_and_add_ligand(pose_p, motif_pose_p, pose_motif, pose_motif, new_pose="/home/markus/projects/rfdiffusion_testing/motif_replacement/D33-C34-B26-A44_0001_0001_replaced.pdb", ligand_chain="F")
#
#


4
5
6
7
8
9
10
21
22
23
24
25
26
27
38
39
40
41
42
43
44
53
54
55
56
57
58
59


'/home/markus/projects/rfdiffusion_testing/motif_replacement/D33-C34-B26-A44_0001_0001_replaced.pdb'